In [17]:
import tensorflow as tf
print(tf.__version__)


2023-09-02 15:56:58.695232: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.13.0


In [26]:
import boto3

# Inicializa el cliente de S3
s3 = boto3.client('s3')

# Nombre del bucket en S3 donde deseas almacenar el modelo
# (nota: los nombres de los buckets deben ser únicos globalmente en AWS)
BUCKET_NAME = 'neuronalnetwork'
MODEL_FILE_NAME = 'model1.tar.gz'
MODEL_PATH_ON_S3 = 'path/on/s3/' + MODEL_FILE_NAME

# Sube el modelo a S3
s3.upload_file(MODEL_FILE_NAME, BUCKET_NAME, MODEL_PATH_ON_S3)

print(f"Modelo {MODEL_FILE_NAME} subido a {BUCKET_NAME}/{MODEL_PATH_ON_S3}")


Modelo model1.tar.gz subido a neuronalnetwork/path/on/s3/model1.tar.gz


In [1]:
from sagemaker.tensorflow import TensorFlowModel

sagemaker_role = 'arn:aws:iam::027331693661:role/Modelv2'

# Crea un modelo TensorFlow en SageMaker a partir de tus artefactos
model = TensorFlowModel(model_data='s3://neuronalnetwork/path/on/s3/model1.tar.gz',
                        role=sagemaker_role, 
                        framework_version='2.13.0')

# Despliega el modelo a un endpoint
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')



KeyboardInterrupt



In [45]:
import pandas as pd

mainpath = "/Users/soldash/Documents/Tesis/python/Datos/Todas/" #ruta de Carga de los datos
filename = "dataframe_concatenado.csv" #Nombre del archivo a cargar
data = pd.read_csv(mainpath+filename, sep=",")
data = data[data["Seizure"] == 1]
data

,ECG,Time,Seizure,RRI,SDNN,HR,HR_std,NN50,pNN50,HF,LF,Total_power,LF_HF
1001,-25.66,2022-07-01 13:24:24.760,1,628,21.847794,92,0.496101,0,0.000000,108.800771,34.126383,249.789057,0.313659
1002,111.64,2022-07-01 13:24:38.080,1,678,41.233267,93,0.574147,4,15.384615,135.113971,17.492237,252.425314,0.129463
1003,-42.29,2022-07-01 13:24:18.120,1,642,19.121367,92,0.489230,0,0.000000,53.146838,24.522385,441.868074,0.461408
1004,-4.06,2022-07-01 13:24:50.140,1,660,120.469669,92,1.299070,13,39.393939,40.416363,149.484771,593.815833,3.698620
1005,19.95,2022-07-01 13:24:34.200,1,654,41.927220,93,0.512531,5,19.230769,168.454661,31.792269,253.399146,0.188729
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55064,39.30,2022-07-02 02:14:36.300,1,1030,207.740191,56,0.657142,10,52.631579,15346.970463,73066.954258,201813.205584,4.761002
55065,55.83,2022-07-02 02:14:53.300,1,1535,223.170339,54,0.644186,12,63.157895,9374.601546,45946.611458,115020.536836,4.901180
55066,-12.27,2022-07-02 02:14:32.950,1,985,211.223116,56,0.566998,9,50.000000,14809.079024,76828.157582,209861.117912,5.187909
55067,16.65,2022-07-02 02:14:35.400,1,1005,223.343463,56,0.621812,11,57.894737,14085.983384,74526.528685,202095.715702,5.290829


In [44]:
import sagemaker
from sagemaker.tensorflow import TensorFlowPredictor
from sklearn.preprocessing import StandardScaler

# Define el nombre del endpoint
endpoint_name = 'tensorflow-inference-2023-09-02-23-00-24-140'  # Cambia esto al nombre real del endpoint

# Crea un predictor
predictor = TensorFlowPredictor(endpoint_name=endpoint_name)

# Usa el predictor para obtener una predicción
#input_data = [618, 25.65814159063469, 97, 0.652648986464887, 2, 6.896551724137931, 12462.459981061436, 9715.511213369073, 41987.7445303172, 0.7795821393314996]
predictors = ['RRI', 'SDNN', 'HR', 'HR_std', 'NN50', 'pNN50', 'HF', 'LF', 'Total_power', 'LF_HF']
X = data[predictors].values

# Escala los datos usando el scaler del entrenamiento
scaler = StandardScaler()
X_scaled = scaler.transform(X)

# Usa el predictor para obtener las predicciones de validación
results = []
for row in X:
    result = predictor.predict(row.tolist())
    results.append(result)
    print(result)

print(results)


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [46]:
import boto3

# Inicializa el cliente de S3
s3 = boto3.client('s3')

# Nombre de tu bucket y archivo
bucket_name = 'neuronalnetwork'
file_name = 'scaler_neuronal_net.pkl'

# Sube el archivo al S3
with open(file_name, "rb") as f:
    s3.upload_fileobj(f, bucket_name, file_name)